### 数据库schema：
数据库sentence.db schema如下：

sentence(sid,sentence1, sentence2, similar_score,sen1_vector,sen2_vector,vecSim_score)

分别是句子id，句子1，句子2，句子相似度得分，句子1嵌入向量，句子2嵌入向量，向量相似度得分）

### 任务要求：

调用嵌入模型：可以使用sqlite-vss或sqlite-vec或调用本地中文嵌入模型（huggingface等拉取），对每个数据库中每个元组的两个句子分别进行向量嵌入，并计算向量相似度得分。更新数据库sentence.db的后三列。

### 输出要求：

按照这样的方法：

```python
df = pd.read_sql_query("""
    SELECT sid, sentence1, sentence2, similar_score, sen1_vector, sen2_vector,vecSim_score FROM sentence
""", conn)
print(df.head(20))
```
输出数据库的前20个元组。但请注意，助教评分时可能会运行你的代码，抽查其他元组结果。

In [5]:
import sqlite3
import os
import pandas as pd

In [ ]:
# 文件路径修改成自己的文件路径
DATA_FILE = 'sts-b-train.txt'
DB_FILE = 'sentence.db'

if os.path.exists(DB_FILE):
    os.remove(DB_FILE)

conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

In [3]:
cursor.execute('''
CREATE TABLE sentence (
    sid INTEGER PRIMARY KEY AUTOINCREMENT,
    sentence1 TEXT NOT NULL,
    sentence2 TEXT NOT NULL,
    similar_score REAL NOT NULL,
    sen1_vector BLOB,
    sen2_vector BLOB,
    vecSim_score REAL
)
''')
conn.commit()

In [4]:
with open(DATA_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) != 3:
            continue
        sentence1, sentence2, score = parts
        cursor.execute('''
            INSERT INTO sentence (sentence1, sentence2, similar_score)
            VALUES (?, ?, ?)
        ''', (sentence1, sentence2, float(score)))

In [7]:
df = pd.read_sql_query("""
    SELECT sid, sentence1, sentence2, similar_score, sen1_vector, sen2_vector,vecSim_score FROM sentence
""", conn)
print(df.head(10))

   sid          sentence1                  sentence2  similar_score  \
0    1          一架飞机要起飞了。                  一架飞机正在起飞。            5.0   
1    2       一个男人在吹一支大笛子。                   一个人在吹长笛。            3.0   
2    3  一个人正把切碎的奶酪撒在比萨饼上。  一个男人正在把切碎的奶酪撒在一块未煮好的比萨饼上。            3.0   
3    4            三个人在下棋。                    两个人在下棋。            2.0   
4    5          一个人在拉大提琴。              一个坐着的人正在拉大提琴。            4.0   
5    6            有些人在战斗。                    两个人在打架。            4.0   
6    7           一个男人在抽烟。                   一个男人在滑冰。            0.0   
7    8            那人在弹钢琴。                    那人在弹吉他。            1.0   
8    9       一个男人在弹吉他和唱歌。        一位女士正在弹着一把原声吉他，唱着歌。            2.0   
9   10    一个人正把一只猫扔到天花板上。             一个人把一只猫扔在天花板上。            5.0   

  sen1_vector sen2_vector vecSim_score  
0        None        None         None  
1        None        None         None  
2        None        None         None  
3        None        None         None  
4        None

## 提示：一些可能的嵌入向量的方式：

### 1.安装 sqlite-vss（基于 FAISS）

```python
!pip install faiss-cpu
!pip install sqlite-vss

import sqlite3
import sqlite_vss
conn = sqlite3.connect("sentence.db")
sqlite_vss.load(conn)
```
### 2.安装sqlite-vec(建议python版本>=3.9)

```python
!pip install sqlite-vec


import sqlite3
import sqlite_vec
conn = sqlite3.connect("sentence.db")
sqlite_vec.load(conn)
```

### 3. 使用hugging_face等拉取嵌入模型

```python
!pip install sentence-transformers
!pip install torch

model = HuggingFaceEmbeddings('YOU MODEL PATH')
```
